In [1]:
from bd3client.CentralService import CentralService
from bd3client.Sensor import Sensor
from bd3client.Timeseries import Timeseries
import os
import json
import arrow
import pandas as pd

In [3]:
with open("config/bd3config.json", "r") as fp:
    configs = json.load(fp)
    hostname = configs['hostname']
    valid_naes = configs['valid_naes']
    base_dir = configs['base_dir']
    begin_time = arrow.get(configs['begin_time']).timestamp
    end_time = arrow.get(configs['end_time']).timestamp
with open("config/bd3secrets.json", "r") as fp:
    cred = json.load(fp)
    cid = cred['cid']
    ckey = cred['ckey']
cs = CentralService(hostname, cid, ckey)
sensor_api = Sensor(cs)
ts_api = Timeseries(cs)

with open("metadata/bacnet_devices.json", "r") as fp:
    bacnet_devices = json.load(fp)

In [6]:
uuid_dict = dict()
i = 0
for nae_name in valid_naes:
    objs = bacnet_devices[nae_name]['objs']
    for obj in objs:
        if obj['data_type'] not in [0,1,2,3,4,5,13,14,19]:
            continue        
        if i%500==0:
            print(i)
        i += 1
        props = obj['props']
        tag_val_dict = {
            'BACNet_DataType': obj['data_type'],
            'BACNet_InstanceNumber':obj['props']['instance'],
            'NAENum':nae_name
            }
        srcid = str(nae_name) + "_" + str(obj['data_type']) + "_" + str(obj['props']['instance'])
        try:
            uuid = sensor_api.search(tag_val_dict)['result'][0]['name']
            uuid_dict[srcid] = uuid
        except:
            print('----------------')
            print(srcid)
            print(sensor_api.search(tag_val_dict))

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500


In [7]:
header = ['value']
index_label="time"
for srcid, uuid in uuid_dict.items():
    try:
        ts = ts_api.getTimeseriesDataPoints(uuid, begin_time, end_time)
        time_list = [arrow.get(row[0]) for row in ts['data']['series'][0]['values']]
        value_list = [row[2] for row in ts['data']['series'][0]['values']]
        filename = base_dir + "/" + srcid + ".csv"
        pd.Series(index=time_list, data=value_list).to_csv(filename, header=header, index_label=index_label)
    except:
        print srcid, uuid
        print ts_api.getTimeseriesDataPoints(uuid, begin_time, end_time)

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-7-5ad2fe4d90d1>, line 11)